# 01. Prepare Sample Files for BGCflow
We are interested to learn a bit more about the BGCs of the two phyla found in the MAGs from [Singleton et al., 2021](https://www.nature.com/articles/s41467-021-22203-2).

These two are:
- [p__Myxococcota](https://gtdb.ecogenomic.org/tree?r=p__Myxococcota) - a group that are talented to have BGCs
- [f__Nitrospiraceae](https://gtdb.ecogenomic.org/tree?r=p__Nitrospirota) - a group that are functionally relevant to the WWTP AS environment

As previous attempt in clustering the MAGs with other BGCs found in MIBIG database did not come to a hit, we are interested if we can find closely related BGCs within other genomes found in public database. For that, we will compare the BGCs detected in the MAGs vs genomes within the same phylum that can be found in GTDB.

This notebook will prepare a sample file (`.csv`) for [BGCflow](https://github.com/NBChub/bgcflow) to run the pipeline for comparative BGC.


## Table of Contents
* [Library and paths setting](#load-library)
* [Getting MAGs information from Singleton 2021](#second-bullet)
    * [Data cleaning](#data-cleaning-mags)
        * [Map to BioProject](#map-to-bioproject)
    * [Filtering for Myxococcales](#myxo-filter-mags)
    * [Filtering for Nitrospira](#nitro-filter-mags)

### Load Library <a class="anchor" id="load-library"></a>


In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
## Useful scripts
def get_gtdb_tax(df):
    """
    Clean bac120_taxonomy_<release>.tsv into pandas dataframe.
    Get the file from https://data.gtdb.ecogenomic.org/releases/release202/202.0/bac120_taxonomy_r202.tsv
    """
    # cleaning
    df_tax = pd.DataFrame(df.apply(lambda x: x.split(";")).to_dict()).T
    df_tax = df_tax.rename(columns={0:"domain",
                          1:"phylum",
                          2:"class",
                          3:"order",
                          4:"family",
                          5:"genus",
                          6:"species"})
    return df_tax

def format_bgcflow(df, outfile):
    """
    Input a filtered Pandas DataFrame from bac120_taxonomy_<release>.tsv and format it to bgcflow samples.csv.
    """
    filtered_df = df.copy()
    for c in filtered_df.columns:
        filtered_df.loc[:, c] = filtered_df.loc[:, c].apply(lambda x: x.split("__")[-1])
    filtered_df.insert(0, "genome_id", [g for g in filtered_df.index]) 
    filtered_df.insert(1, "source", ["ncbi" for g in filtered_df.index]) 
    filtered_df.insert(2, "organism", filtered_df.loc[:, "species"])
    filtered_df.loc[:, "species"] = filtered_df.loc[:, "species"].apply(lambda x: x.split(" ")[-1])
    filtered_df.loc[:, "strain"] = np.nan
    filtered_df.loc[:, "closest_placement_reference"] = np.nan
    filtered_df.to_csv(outfile, index=False)
    return filtered_df

In [ ]:
# Set paths
tables = "../tables"
data = "../data"

# Getting MAGs information from Singleton 2021 <a class="anchor" id="load-library">
## Data Cleaning <a class="anchor" id="data-cleaning-mags">
First we will get the metadata from the paper and map them to NCBI assembly ids. Grab the metadata with:

In [ ]:
! wget -P {data} https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-021-22203-2/MediaObjects/41467_2021_22203_MOESM5_ESM.xlsx -nc

In [ ]:
# Load supplementary materials
df_MAGs = pd.read_excel(os.path.join(data, "41467_2021_22203_MOESM5_ESM.xlsx"), skiprows=1)
df_MAGs.head(1)

In [ ]:
# What columns are available?
df_MAGs.columns

## Map to BioProject</a><a class="anchor" id="map-to-bioproject">
From the table above, we can see that it contains NCBI accession number, but we want the assembly ids (refseq or genbank). We can grab the information from bioproject. The easiest way is to go to https://www.ncbi.nlm.nih.gov/bioproject/prjna629478 and download the assembly details (we've downloaded it in this repository).

In [ ]:
df_bioproject = pd.read_csv(os.path.join(data, "PRJNA629478_AssemblyDetails.txt"), skiprows=1, sep="\t", index_col=False)
df_bioproject.head(1)

In [ ]:
df_bioproject.shape

Looking at the structure, we can map the two tables using Isolate name in df_bioproject and MAG in df_MAGs:

In [ ]:
df_MAGs.loc[:, "Isolate"] = [b.strip(".fa") for b in df_MAGs.MAG]
df_MAGs.head(1)

In [ ]:
df_MAGs_bioproject = pd.merge(df_MAGs, df_bioproject, on='Isolate')

In [ ]:
## Format to BGCflow input
df_MAGs_bioproject.columns

In [ ]:
# Clean 
df_MAGs_bioproject = df_MAGs_bioproject.rename(columns={'# Assembly' : 'genome_id'})
df_MAGs_bioproject = df_MAGs_bioproject.set_index("genome_id", drop=False)
df_MAGs_bioproject.to_csv("../tables/df_MAGs_bioproject.csv", index=False)
df_MAGs_bioproject.head(1)

In [ ]:
# Format into bgcflow input table
df_MAGs_bgcflow = format_bgcflow(get_gtdb_tax(df_MAGs_bioproject["GTDBTax"]), os.path.join(tables, "df_MAGs_bgcflow.csv"))
df_MAGs_bgcflow.head(1)

## Filtering for p__Myxococcota MAGs </a><a class="anchor" id="myxo-filter-mags"></a>

In [ ]:
df_p__Myxococcota_MAGs = df_MAGs_bioproject[df_MAGs_bioproject.loc[:, "GTDBTax"].str.contains("p__Myxococcota")]
df_p__Myxococcota_MAGs.shape

In [ ]:
df_p__Myxococcota_MAGs.loc[:, "GTDBTax"].unique()

In [ ]:
format_bgcflow(get_gtdb_tax(df_p__Myxococcota_MAGs["GTDBTax"]), os.path.join(tables, "p__Myxococcota_MAGs.csv")).head(1)

## Filtering for p__Nitrospirota MAGs </a><a class="anchor" id="nitro-filter-mags"></a>

In [ ]:
df_p__Nitrospirota_MAGs = df_MAGs_bioproject[df_MAGs_bioproject.loc[:, "GTDBTax"].str.contains("p__Nitrospirota")]
df_p__Nitrospirota_MAGs.shape

In [ ]:
df_p__Nitrospirota_MAGs.loc[:, "GTDBTax"].unique()

In [ ]:
df_p__Nitrospirota_MAGs = format_bgcflow(get_gtdb_tax(df_p__Nitrospirota_MAGs["GTDBTax"]), os.path.join(tables, "p__Nitrospirota_MAGs.csv"))

# Getting genome assembly accession id from GTDB
If you hadn't done so, download the taxonomy mapping from the latest GTDB release:

In [ ]:
# Download bacteria taxonomy from release 202
! wget -P {data} https://data.gtdb.ecogenomic.org/releases/release202/202.0/bac120_taxonomy_r202.tsv -nc

In [ ]:
df = pd.read_csv(os.path.join(data, "bac120_taxonomy_r202.tsv"), sep="\t", header=None, index_col=0)
df.index = [i.split("_",1)[-1] for i in df.index]
# Clean the file
df_tax = get_gtdb_tax(df[1])
df_tax.head(1)

## Get all p__Myxococcota from MAGs and GTDB

In [ ]:
p__Myxococcota_MAGs = df_MAGs_bioproject[df_MAGs_bioproject.loc[:, "GTDBTax"].str.contains("p__Myxococcota")]
p__Myxococcota_MAGs.shape

In [ ]:
p__Myxococcota_GTDB = df_tax[df_tax.loc[:, "phylum"] == "p__Myxococcota"]
p__Myxococcota_GTDB.shape

In [ ]:
df_p__Myxococcota_gtdb = format_bgcflow(p__Myxococcota_GTDB, os.path.join(tables, "p__Myxococcota_gtdb.csv"))

In [ ]:
df_p__Myxococcota_MAGs = format_bgcflow(get_gtdb_tax(p__Myxococcota_MAGs["GTDBTax"]), os.path.join(tables, "p__Myxococcota_MAGs.csv"))

In [ ]:
df_p__Myxococcota_all = pd.concat([df_p__Myxococcota_gtdb, df_p__Myxococcota_MAGs])
df_p__Myxococcota_all.to_csv("../tables/p__Myxococcota_all.csv")
df_p__Myxococcota_all.shape

## Get all p__Nitrospirota from MAGs and GTDB

In [ ]:
p__Nitrospirota_MAGs = df_MAGs_bioproject[df_MAGs_bioproject.loc[:, "GTDBTax"].str.contains("p__Nitrospirota")]
p__Nitrospirota_MAGs.shape

In [ ]:
p__Nitrospirota_GTDB = df_tax[df_tax.loc[:, "phylum"] == "p__Nitrospirota"]
p__Nitrospirota_GTDB.shape

In [ ]:
df_p__Nitrospirota_gtdb = format_bgcflow(p__Nitrospirota_GTDB, os.path.join(tables, "p__Nitrospirota_gtdb.csv"))
df_p__Nitrospirota_MAGs = format_bgcflow(get_gtdb_tax(p__Nitrospirota_MAGs["GTDBTax"]), os.path.join(tables, "p__Nitrospirota_MAGs.csv"))

In [ ]:
df_p__Nitrospirota_all = pd.concat([df_p__Nitrospirota_gtdb, df_p__Nitrospirota_MAGs])
df_p__Nitrospirota_all.to_csv("../tables/p__Nitrospirota_all.csv")

## New Project - HQ Nitrospiraceae
We would also try zooming in for the HQ genomes available in NCBI

In [ ]:
df_raw_HQ_Nitrospiraceae = pd.read_csv("../data/assembly_result_Nitrospiraceae_hq.txt", sep="\t")
df_raw_HQ_Nitrospiraceae = df_raw_HQ_Nitrospiraceae.fillna("")

In [ ]:
# Check if genbank available in GTDB

ncbi_in_GTDB = []
ncbi_not_in_GTDB = []

for i in df_raw_HQ_Nitrospiraceae.index:
    refseq = df_raw_HQ_Nitrospiraceae.loc[i, "RefSeq Assembly ID (Accession.version)"]
    genbank = df_raw_HQ_Nitrospiraceae.loc[i, "GenBank Assembly ID (Accession.version)"]
    
    if refseq in df_p__Nitrospirota_all.genome_id:
        print(refseq, "refseq in dataframe")
    elif genbank in df_p__Nitrospirota_all.genome_id:
        print(genbank, "genbank in dataframe")
    
    elif refseq in df_tax.index:
        print(refseq, "refseq in GTDB:", f"adding {refseq} to list...")
        ncbi_in_GTDB.append(refseq)
    elif genbank in df_tax.index:
        print(genbank, "genbank in GTDB:", f"adding {genbank} to list...")
        ncbi_in_GTDB.append(genbank)
    
    else:
        if refseq != "":
            print(refseq, "not found:", f"adding {refseq} to list...")
            ncbi_not_in_GTDB.append(refseq)
        else:
            print(genbank, "not found:", f"adding {genbank} to list...")
            ncbi_not_in_GTDB.append(genbank)
        
ncbi_not_in_GTDB

In [ ]:
p__Nitrospiraceae_NCBI_HQ = df_tax.loc[ncbi_in_GTDB, :]
df_p__Nitrospiraceae_NCBI_HQ = format_bgcflow(p__Nitrospiraceae_NCBI_HQ, os.path.join(tables, "f__Nitrospiraceae_NCBI_HQ.csv"))
for i in ncbi_not_in_GTDB:
    df_p__Nitrospiraceae_NCBI_HQ.loc[i, ["genome_id", "source"]] = [i, "ncbi"]
df_p__Nitrospiraceae_NCBI_HQ.to_csv(os.path.join(tables, "f__Nitrospiraceae_NCBI_HQ.csv"), index=False)

In [ ]:
df_p__Nitrospiraceae_HQ_all = pd.concat([df_p__Nitrospiraceae_NCBI_HQ, df_p__Nitrospirota_MAGs])
df_p__Nitrospiraceae_HQ_all.to_csv("../tables/f__Nitrospiraceae_HQ_all.csv")

## Summary

The results are stored in `../tables`, and can be used as the sample file for BGCflow